https://mlwhiz.com/blog/2019/02/19/siver_medal_kaggle_learnings/

https://towardsdatascience.com/https-medium-com-tanaygahlot-moving-beyond-the-distributional-model-for-word-representation-b0823f1769f8

In [0]:
from __future__ import absolute_import, division

import os
import time
import numpy as np
import pandas as pd
import gensim
from tqdm import tqdm
from nltk.stem import PorterStemmer
ps = PorterStemmer()
from nltk.stem.lancaster import LancasterStemmer
lc = LancasterStemmer()
from nltk.stem import SnowballStemmer
sb = SnowballStemmer("english")
import gc

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate, LSTM, GRU
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers

import sys
from os.path import dirname
#sys.path.append(dirname(dirname(__file__)))
from keras import initializers
from keras.engine import InputSpec, Layer
from keras import backend as K

import spacy


In [0]:
# https://github.com/bfelbo/DeepMoji/blob/master/deepmoji/attlayer.py
class AttentionWeightedAverage(Layer):
    """
    Computes a weighted average of the different channels across timesteps.
    Uses 1 parameter pr. channel to compute the attention value for a single timestep.
    """

    def __init__(self, return_attention=False, **kwargs):
        self.init = initializers.get('uniform')
        self.supports_masking = True
        self.return_attention = return_attention
        super(AttentionWeightedAverage, self).__init__(** kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(ndim=3)]
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[2], 1),
                                 name='{}_W'.format(self.name),
                                 initializer=self.init)
        self.trainable_weights = [self.W]
        super(AttentionWeightedAverage, self).build(input_shape)

    def call(self, x, mask=None):
        # computes a probability distribution over the timesteps
        # uses 'max trick' for numerical stability
        # reshape is done to avoid issue with Tensorflow
        # and 1-dimensional weights
        logits = K.dot(x, self.W)
        x_shape = K.shape(x)
        logits = K.reshape(logits, (x_shape[0], x_shape[1]))
        ai = K.exp(logits - K.max(logits, axis=-1, keepdims=True))

        # masked timesteps have zero weight
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            ai = ai * mask
        att_weights = ai / (K.sum(ai, axis=1, keepdims=True) + K.epsilon())
        weighted_input = x * K.expand_dims(att_weights)
        result = K.sum(weighted_input, axis=1)
        if self.return_attention:
            return [result, att_weights]
        return result

    def get_output_shape_for(self, input_shape):
        return self.compute_output_shape(input_shape)

    def compute_output_shape(self, input_shape):
        output_len = input_shape[2]
        if self.return_attention:
            return [(input_shape[0], output_len), (input_shape[0], input_shape[1])]
        return (input_shape[0], output_len)

    def compute_mask(self, input, input_mask=None):
        if isinstance(input_mask, list):
            return [None] * len(input_mask)
        else:
            return None

In [3]:
# https://www.kaggle.com/cpmpml/spell-checker-using-word2vec
spell_model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/My Drive/Colab Notebooks/wiki-news-300d-1M.vec')
words = spell_model.index2word
w_rank = {}
for i,word in enumerate(words):
    w_rank[word] = i
WORDS = w_rank

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
def words(text):
   return re.findall(r'\w+', text.lower())
def P(word): 
    "Probability of `word`."
    # use inverse of rank as proxy
    # returns 0 if the word isn't in the dictionary
    return - WORDS.get(word, 0)
def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)
def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or [word])
def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)
def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))
def singlify(word):
    return "".join([letter for i,letter in enumerate(word) if i == 0 or letter != word[i-1]])


In [0]:
# modified version of 
# https://www.kaggle.com/sudalairajkumar/a-look-at-different-embeddings
# https://www.kaggle.com/danofer/different-embeddings-with-attention-fork
# https://www.kaggle.com/shujian/different-embeddings-with-attention-fork-fork
def load_glove(word_dict, lemma_dict):
    EMBEDDING_FILE = '/content/drive/My Drive/Colab Notebooks/glove.840B.300d.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))
    embed_size = 300
    nb_words = len(word_dict)+1
    embedding_matrix = np.zeros((nb_words, embed_size), dtype=np.float32)
    unknown_vector = np.zeros((embed_size,), dtype=np.float32) - 1.
    print(unknown_vector[:5])
    for key in tqdm(word_dict):
        word = key
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = key.lower()
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = key.upper()
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = key.capitalize()
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = ps.stem(key)
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = lc.stem(key)
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = sb.stem(key)
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = lemma_dict[key]
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        if len(key) > 1:
            word = correction(key)
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                embedding_matrix[word_dict[key]] = embedding_vector
                continue
        embedding_matrix[word_dict[key]] = unknown_vector                    
    return embedding_matrix, nb_words 


def load_fasttext(word_dict, lemma_dict):
    EMBEDDING_FILE = '/content/drive/My Drive/Colab Notebooks/paragram_300_sl999.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o)>100)
    embed_size = 300
    nb_words = len(word_dict)+1
    embedding_matrix = np.zeros((nb_words, embed_size), dtype=np.float32)
    unknown_vector = np.zeros((embed_size,), dtype=np.float32) - 1.
    print(unknown_vector[:5])
    for key in tqdm(word_dict):
        word = key
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = key.lower()
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = key.upper()
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = key.capitalize()
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = ps.stem(key)
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = lc.stem(key)
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = sb.stem(key)
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = lemma_dict[key]
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        if len(key) > 1:
            word = correction(key)
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                embedding_matrix[word_dict[key]] = embedding_vector
                continue
        embedding_matrix[word_dict[key]] = unknown_vector                    
    return embedding_matrix, nb_words 


def load_para(word_dict, lemma_dict):
    EMBEDDING_FILE = '/content/drive/My Drive/Colab Notebooks/paragram_300_sl999.txt'
    def get_coefs(word,*arr): 
      return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') if len(o)>100)
    embed_size = 300
    nb_words = len(word_dict)+1
    embedding_matrix = np.zeros((nb_words, embed_size), dtype=np.float32)
    unknown_vector = np.zeros((embed_size,), dtype=np.float32) - 1.
    print(unknown_vector[:5])
    for key in tqdm(word_dict):
        word = key
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = key.lower()
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = key.upper()
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = key.capitalize()
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = ps.stem(key)
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = lc.stem(key)
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = sb.stem(key)
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = lemma_dict[key]
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        if len(key) > 1:
            word = correction(key)
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                embedding_matrix[word_dict[key]] = embedding_vector
                continue
        embedding_matrix[word_dict[key]] = unknown_vector                    
    return embedding_matrix, nb_words 


In [0]:
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    

class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [0]:
def build_model(embedding_matrix, nb_words, embedding_size=300):
    inp = Input(shape=(max_length,))
    x = Embedding(nb_words, embedding_size, weights=[embedding_matrix], trainable=False)(inp)
    x = SpatialDropout1D(0.3)(x)
    x1 = Bidirectional(CuDNNLSTM(256, return_sequences=True))(x)
    x2 = Bidirectional(CuDNNGRU(128, return_sequences=True))(x1)
    max_pool1 = GlobalMaxPooling1D()(x1)
    max_pool2 = GlobalMaxPooling1D()(x2)
    conc = Concatenate()([max_pool1, max_pool2])
    predictions = Dense(1, activation='sigmoid')(conc)
    model = Model(inputs=inp, outputs=predictions)
    adam = optimizers.Adam(lr=learning_rate)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [0]:
def model_lstm_atten(embedding_matrix, nb_words, embedding_size=300):
    inp = Input(shape=(max_length,))
    x = Embedding(nb_words, embedding_size, weights=[embedding_matrix], trainable=False)(inp)
    x = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x)
    x = Bidirectional(CuDNNLSTM(64, return_sequences=True))(x)
    x = AttentionWithContext()(x)
    x = Dense(64, activation="relu")(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [59]:
start_time = time.time()
print("Loading data ...")
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/train.csv').fillna(' ')
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/test.csv').fillna(' ')
train_text = train['text']
test_text = test['text']
text_list = pd.concat([train_text, test_text])
y = train['target'].values
num_train_data = y.shape[0]
print("--- %s seconds ---" % (time.time() - start_time))


Loading data ...
--- 0.06915020942687988 seconds ---


In [52]:
import matplotlib.pyplot as plt
train['word_len']=train['text'].apply(lambda x:len(x))
test['word_len']=test['text'].apply(lambda x:len(x))
print("mean : ",train['word_len'].mean())
print("median : ",train['word_len'].median())
print("mean : ",test['word_len'].mean())
print("median : ",test['word_len'].median())
train['word_len']=train['text'].apply(lambda x:len(x))
# print(train['word_len'].quantile(0.10))
# print(train['word_len'].quantile(0.20))
# print(train['word_len'].quantile(0.30))
# print(train['word_len'].quantile(0.40))
# print(train['word_len'].quantile(0.50))
# print(train['word_len'].quantile(0.60))
# print(train['word_len'].quantile(0.70))
# print(train['word_len'].quantile(0.80))
# print(train['word_len'].quantile(0.90))

mean :  101.03743596479706
median :  107.0
mean :  102.10818265399939
median :  109.0


In [0]:
# !python -m spacy download en_core_web_lg

In [13]:
start_time = time.time()
print("Spacy NLP ...")
nlp = spacy.load('/content/drive/My Drive/Colab Notebooks/en_core_web_lg-2.1.0/en_core_web_lg/en_core_web_lg-2.1.0', disable=['parser','ner','tagger'])
nlp.vocab.add_flag(lambda s: s.lower() in spacy.lang.en.stop_words.STOP_WORDS, spacy.attrs.IS_STOP)
word_dict = {}
word_index = 1
lemma_dict = {}
docs = nlp.pipe(text_list, n_threads = 2)
word_sequences = []
for doc in tqdm(docs):
    word_seq = []
    for token in doc:
        if (token.text not in word_dict) and (token.pos_ is not "PUNCT"):
            word_dict[token.text] = word_index
            word_index += 1
            lemma_dict[token.text] = token.lemma_
        if token.pos_ is not "PUNCT":
            word_seq.append(word_dict[token.text])
    word_sequences.append(word_seq)
del docs
gc.collect()
train_word_sequences = word_sequences[:num_train_data]
test_word_sequences = word_sequences[num_train_data:]
print("--- %s seconds ---" % (time.time() - start_time))

Spacy NLP ...


10876it [00:04, 2268.18it/s]


--- 41.63713026046753 seconds ---


In [24]:
# hyperparameters
max_length = 140
embedding_size = 600
learning_rate = 0.001
batch_size = 512
num_epoch = 4

train_word_sequences = pad_sequences(train_word_sequences, maxlen=max_length, padding='post')
test_word_sequences = pad_sequences(test_word_sequences, maxlen=max_length, padding='post')
print(train_word_sequences[:10])
print(test_word_sequences[:10])
pred_prob = np.zeros((len(test_word_sequences),), dtype=np.float32)

[[  1   2   3 ...   0   0   0]
 [ 15  16  17 ...   0   0   0]
 [ 23  24  25 ...   0   0   0]
 ...
 [ 86  87  88 ...   0   0   0]
 [ 96  97  98 ...   0   0   0]
 [ 86  87 105 ...   0   0   0]]
[[  46  762   57 ...    0    0    0]
 [8961  461    8 ...    0    0    0]
 [ 630  108   57 ...    0    0    0]
 ...
 [1697  168  323 ...    0    0    0]
 [ 157   57 2500 ...    0    0    0]
 [7827  972  168 ...    0    0    0]]


In [15]:
start_time = time.time()
print("Loading embedding matrix ...")
embedding_matrix_glove, nb_words = load_glove(word_dict, lemma_dict)
embedding_matrix_fasttext, nb_words = load_fasttext(word_dict, lemma_dict)
embedding_matrix = np.concatenate((embedding_matrix_glove, embedding_matrix_fasttext), axis=1)
print("--- %s seconds ---" % (time.time() - start_time))

Loading embedding matrix ...


  3%|▎         | 1081/34498 [00:00<00:03, 10754.32it/s]

[-1. -1. -1. -1. -1.]


  4%|▎         | 1253/34498 [00:00<00:02, 12505.53it/s]

[-1. -1. -1. -1. -1.]


100%|██████████| 34498/34498 [00:06<00:00, 5416.87it/s]

--- 205.25434064865112 seconds ---


In [16]:
start_time = time.time()
print("Start training ...")
model = model_lstm_atten(embedding_matrix, nb_words, embedding_size)
model.fit(train_word_sequences, y, batch_size=batch_size, epochs=num_epoch-1, verbose=2)
pred_prob += 0.15*np.squeeze(model.predict(test_word_sequences, batch_size=batch_size, verbose=2))
model.fit(train_word_sequences, y, batch_size=batch_size, epochs=1, verbose=2)
pred_prob += 0.35*np.squeeze(model.predict(test_word_sequences, batch_size=batch_size, verbose=2))
del model, embedding_matrix_fasttext, embedding_matrix
gc.collect()
K.clear_session()
print("--- %s seconds ---" % (time.time() - start_time))

Start training ...











Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/3
 - 10s - loss: 0.6170 - acc: 0.6585
Epoch 2/3
 - 3s - loss: 0.4536 - acc: 0.7955
Epoch 3/3
 - 3s - loss: 0.3950 - acc: 0.8273
Epoch 1/1
 - 3s - loss: 0.3610 - acc: 0.8482


--- 28.643392086029053 seconds ---


In [76]:
start_time = time.time()
print("Loading embedding matrix ...")
embedding_matrix_fasttext, nb_words = load_fasttext(word_dict, lemma_dict)
embedding_matrix_para, nb_words = load_para(word_dict, lemma_dict)
# embedding_matrix = np.concatenate((embedding_matrix_glove, embedding_matrix_para), axis=1)
embedding_matrix = np.concatenate((0.60*embedding_matrix_glove, 0.40*embedding_matrix_fasttext), axis=1)

print("--- %s seconds ---" % (time.time() - start_time))


Loading embedding matrix ...


  4%|▍         | 1304/34498 [00:00<00:02, 13007.57it/s]

[-1. -1. -1. -1. -1.]


  4%|▍         | 1357/34498 [00:00<00:02, 13547.05it/s]

[-1. -1. -1. -1. -1.]


100%|██████████| 34498/34498 [00:05<00:00, 6385.67it/s]


--- 20.552839279174805 seconds ---


In [77]:
embedding_matrix.shape

(34499, 600)

In [0]:
# start_time = time.time()
# print("Start training ...")
model = model_lstm_atten(embedding_matrix, nb_words, embedding_size)
# model.fit(train_word_sequences, y, batch_size=batch_size, epochs=num_epoch-1, verbose=2)
# pred_prob += 0.15*np.squeeze(model.predict(test_word_sequences, batch_size=batch_size, verbose=2))
# model.fit(train_word_sequences, y, batch_size=batch_size, epochs=1, verbose=2)
# pred_prob += 0.35*np.squeeze(model.predict(test_word_sequences, batch_size=batch_size, verbose=2))
# print("--- %s seconds ---" % (time.time() - start_time))

In [79]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
# from sklearn import model_selection   
# kfold=model_selection.KFold
skf = StratifiedKFold(n_splits=4)
skf.get_n_splits(train_word_sequences, y)
cvscores = []
i=1
for train_1, test_1 in skf.split(train_word_sequences, y):
  model = model_lstm_atten(embedding_matrix, nb_words, embedding_size)
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	# Fit the model
  print(model.fit(train_word_sequences[train_1], y[train_1], epochs=15, batch_size=512, verbose=2, validation_data=(train_word_sequences[test_1], y[test_1])))
	# evaluate the model
  # scores = model.evaluate(test_word_sequences[test], y[test], verbose=2)
  # print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
  cvscores.append(scores[1] * 100)
  print(i, "Fold Completed: ",   "--X-- "*25)
  i=i+1
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Train on 5709 samples, validate on 1904 samples
Epoch 1/15
 - 10s - loss: 0.6123 - acc: 0.6674 - val_loss: 0.4892 - val_acc: 0.7815
Epoch 2/15
 - 3s - loss: 0.4654 - acc: 0.7942 - val_loss: 0.4669 - val_acc: 0.7847
Epoch 3/15
 - 3s - loss: 0.4163 - acc: 0.8168 - val_loss: 0.4850 - val_acc: 0.7883
Epoch 4/15
 - 3s - loss: 0.4007 - acc: 0.8255 - val_loss: 0.4540 - val_acc: 0.7962
Epoch 5/15
 - 3s - loss: 0.3878 - acc: 0.8355 - val_loss: 0.4550 - val_acc: 0.7931
Epoch 6/15
 - 3s - loss: 0.3740 - acc: 0.8397 - val_loss: 0.4492 - val_acc: 0.7952
Epoch 7/15
 - 3s - loss: 0.3498 - acc: 0.8527 - val_loss: 0.4824 - val_acc: 0.7925
Epoch 8/15
 - 3s - loss: 0.3324 - acc: 0.8628 - val_loss: 0.4913 - val_acc: 0.7862
Epoch 9/15
 - 3s - loss: 0.3180 - acc: 0.8723 - val_loss: 0.4863 - val_acc: 0.7836
Epoch 10/15
 - 3s - loss: 0.3018 - acc: 0.8774 - val_loss: 0.4934 - val_acc: 0.7873
Epoch 11/15
 - 3s - loss: 0.2954 - acc: 0.8821 - val_loss: 0.5529 - val_acc: 0.7710
Epoch 12/15
 - 3s - loss: 0.2937 - a

In [95]:
pred_prob=model.predict(test_word_sequences, batch_size=batch_size, verbose=2)
f1_score(original['target'], submission['target'])

0.7456238361266293

In [0]:
submission = pd.DataFrame.from_dict({'id': test['id']})
pred_prob=model.predict(test_word_sequences, batch_size=batch_size, verbose=2)
submission['target'] = np.array(pred_prob.round())
submission.to_csv('submission.csv', index=False)
original=pd.read_csv("/content/drive/My Drive/Colab Notebooks/NLP Original.csv")
original['target'].value_counts()

In [92]:
submission['target'].value_counts()

0.0    1980
1.0    1283
Name: target, dtype: int64

In [97]:
def model_lstm_atten(embedding_matrix, nb_words, embedding_size=300):
    inp = Input(shape=(max_length,))
    x = Embedding(nb_words, embedding_size, weights=[embedding_matrix], trainable=False)(inp)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    x = AttentionWithContext()(x)
    x = Dense(64, activation="relu")(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

skf = StratifiedKFold(n_splits=4)
skf.get_n_splits(train_word_sequences, y)
cvscores = []
i=1
for train_1, test_1 in skf.split(train_word_sequences, y):
  model = model_lstm_atten(embedding_matrix, nb_words, embedding_size)
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	# Fit the model
  print(model.fit(train_word_sequences[train_1], y[train_1], epochs=15, batch_size=512, verbose=2, validation_data=(train_word_sequences[test_1], y[test_1])))
	# evaluate the model
  # scores = model.evaluate(test_word_sequences[test], y[test], verbose=2)
  # print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
  cvscores.append(scores[1] * 100)
  print(i, "Fold Completed: ",   "--X-- "*25)
  i=i+1
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Train on 5709 samples, validate on 1904 samples
Epoch 1/15
 - 28s - loss: 0.6389 - acc: 0.6257 - val_loss: 0.5758 - val_acc: 0.7489
Epoch 2/15
 - 17s - loss: 0.4787 - acc: 0.7846 - val_loss: 0.4487 - val_acc: 0.7894
Epoch 3/15
 - 17s - loss: 0.4242 - acc: 0.8122 - val_loss: 0.4462 - val_acc: 0.7978
Epoch 4/15
 - 17s - loss: 0.3877 - acc: 0.8317 - val_loss: 0.4531 - val_acc: 0.7967
Epoch 5/15
 - 17s - loss: 0.3717 - acc: 0.8443 - val_loss: 0.4525 - val_acc: 0.7952
Epoch 6/15
 - 17s - loss: 0.3568 - acc: 0.8506 - val_loss: 0.4718 - val_acc: 0.7920
Epoch 7/15
 - 17s - loss: 0.3459 - acc: 0.8544 - val_loss: 0.4686 - val_acc: 0.7899
Epoch 8/15
 - 17s - loss: 0.3401 - acc: 0.8543 - val_loss: 0.4778 - val_acc: 0.7784
Epoch 9/15
 - 17s - loss: 0.3413 - acc: 0.8574 - val_loss: 0.4742 - val_acc: 0.7904
Epoch 10/15
 - 17s - loss: 0.3178 - acc: 0.8706 - val_loss: 0.5434 - val_acc: 0.7847
Epoch 11/15
 - 17s - loss: 0.2942 - acc: 0.8797 - val_loss: 0.5057 - val_acc: 0.7831
Epoch 12/15
 - 17s - loss:

KeyboardInterrupt: ignored

In [99]:
pred_prob=model.predict(test_word_sequences, batch_size=batch_size, verbose=2)
f1_score(original['target'], submission['target'])

0.7456238361266293

In [100]:
def model_lstm_atten(embedding_matrix, nb_words, embedding_size=300):
    inp = Input(shape=(max_length,))
    x = Embedding(nb_words, embedding_size, weights=[embedding_matrix], trainable=False)(inp)
    x = Bidirectional(LSTM(100, return_sequences=True))(x)
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    x = AttentionWithContext()(x)
    x = Dense(48, activation="relu")(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

skf = StratifiedKFold(n_splits=4)
skf.get_n_splits(train_word_sequences, y)
cvscores = []
i=1
for train_1, test_1 in skf.split(train_word_sequences, y):
  model = model_lstm_atten(embedding_matrix, nb_words, embedding_size)
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	# Fit the model
  print(model.fit(train_word_sequences[train_1], y[train_1], epochs=15, batch_size=512, verbose=2, validation_data=(train_word_sequences[test_1], y[test_1])))
	# evaluate the model
  # scores = model.evaluate(test_word_sequences[test], y[test], verbose=2)
  # print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
  cvscores.append(scores[1] * 100)
  print(i, "Fold Completed: ",   "--X-- "*25)
  i=i+1
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Train on 5709 samples, validate on 1904 samples
Epoch 1/15
 - 30s - loss: 0.6434 - acc: 0.6274 - val_loss: 0.5683 - val_acc: 0.7600
Epoch 2/15
 - 17s - loss: 0.5003 - acc: 0.7761 - val_loss: 0.4748 - val_acc: 0.7857
Epoch 3/15
 - 17s - loss: 0.4202 - acc: 0.8164 - val_loss: 0.4911 - val_acc: 0.7857
Epoch 4/15
 - 17s - loss: 0.4011 - acc: 0.8206 - val_loss: 0.5142 - val_acc: 0.7789
Epoch 5/15
 - 17s - loss: 0.3919 - acc: 0.8297 - val_loss: 0.4590 - val_acc: 0.7925
Epoch 6/15
 - 17s - loss: 0.3683 - acc: 0.8431 - val_loss: 0.4494 - val_acc: 0.7952
Epoch 7/15
 - 17s - loss: 0.3545 - acc: 0.8508 - val_loss: 0.4579 - val_acc: 0.7946
Epoch 8/15
 - 17s - loss: 0.3393 - acc: 0.8592 - val_loss: 0.4627 - val_acc: 0.7920
Epoch 9/15
 - 17s - loss: 0.3357 - acc: 0.8625 - val_loss: 0.4717 - val_acc: 0.7878
Epoch 10/15
 - 17s - loss: 0.3243 - acc: 0.8721 - val_loss: 0.5172 - val_acc: 0.7810
Epoch 11/15
 - 17s - loss: 0.3144 - acc: 0.8704 - val_loss: 0.5191 - val_acc: 0.7778
Epoch 12/15
 - 17s - loss:

In [101]:
pred_prob=model.predict(test_word_sequences, batch_size=batch_size, verbose=2)
f1_score(original['target'], submission['target'])

0.7456238361266293